In [75]:
print('all ok')

all ok


In [76]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [77]:
from langchain_groq import ChatGroq
model = ChatGroq(model="qwen/qwen3-32b")

In [78]:
model.invoke("Hello, whats your name?")

AIMessage(content="<think>\nI need to introduce myself in a friendly and professional manner. When users ask about my name, I should clearly state my name, Qwen, and briefly mention my capabilities as a large language model developed by Alibaba Cloud's Tongyi Lab. I should keep the response concise while conveying my main functions and purpose to help users understand what I can do for them. At the same time, I should maintain a natural and approachable tone rather than being overly formal or technical. I need to make sure the response is in English since the user asked in English. I should also keep the response relatively short, not too lengthy, while still conveying the necessary information. The user is likely looking for a straightforward answer about my identity and capabilities, so I should focus on that without adding unnecessary details.\n</think>\n\nHello! My name is Qwen. I'm a large language model developed by Alibaba Cloud's Tongyi Lab. How can I assist you today?", additi

In [79]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [80]:
#Gemini Embedding Model
embedding_model= GoogleGenerativeAIEmbeddings(model="gemini-embedding-001",google_api_key=os.getenv("GEMINI_API_KEY"))

In [81]:
#HuggingFace Embedding Model
from sentence_transformers import SentenceTransformer
Hugging_embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [82]:
#embedding_model.embed_query("Hello world")
Hugging_embedding_model.encode("Hello world")

array([-3.44772749e-02,  3.10231782e-02,  6.73497003e-03,  2.61089858e-02,
       -3.93620245e-02, -1.60302445e-01,  6.69240132e-02, -6.44148979e-03,
       -4.74504791e-02,  1.47588560e-02,  7.08752796e-02,  5.55276312e-02,
        1.91933345e-02, -2.62513123e-02, -1.01095429e-02, -2.69404557e-02,
        2.23074611e-02, -2.22266484e-02, -1.49692640e-01, -1.74930077e-02,
        7.67625542e-03,  5.43522425e-02,  3.25439707e-03,  3.17258909e-02,
       -8.46213847e-02, -2.94060130e-02,  5.15956134e-02,  4.81240600e-02,
       -3.31482221e-03, -5.82791679e-02,  4.19692732e-02,  2.22106855e-02,
        1.28188834e-01, -2.23389715e-02, -1.16563160e-02,  6.29283935e-02,
       -3.28763351e-02, -9.12260413e-02, -3.11753470e-02,  5.26994914e-02,
        4.70348299e-02, -8.42031166e-02, -3.00561991e-02, -2.07448304e-02,
        9.51783545e-03, -3.72179062e-03,  7.34328525e-03,  3.93243879e-02,
        9.32740644e-02, -3.78859648e-03, -5.27420677e-02, -5.80581613e-02,
       -6.86436146e-03,  

## Data Ingestion

In [83]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [84]:
import os
file_path = os.path.join(os.getcwd(),"data","sample.pdf")

In [85]:
loader = PyPDFLoader(file_path=file_path)

In [86]:
documents = loader.load()
len(documents)

77

# this is a experimental thing there is no deterministic way to split the text

In [87]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100 , length_function=len)

In [88]:
docs = text_splitter.split_documents(documents)
len(docs)
# 662 Documents available after chunking

662

In [89]:
#docs[0].metadata
#docs[0].page_content

In [90]:
from langchain.vectorstores import FAISS

In [91]:
from langchain.embeddings import HuggingFaceEmbeddings

hug_embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

In [92]:
#embedlen=hug_embedding_model.embed_documents(docs[0].page_content)
#len(embedlen)

In Memory storage

In [93]:
vector_store = FAISS.from_documents(docs, hug_embedding_model)

## Retrieval process - from the vector database , going to fetch or retrieve most relevant documents/ Ranked results

In [94]:
relevant_results = vector_store.similarity_search("llama2 finetuning benchmark experiments?")
relevant_results

[Document(id='ae06a9b0-3328-4710-a594-a0586a340258', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'e:\\LLMProjects\\documentportal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 69, 'page_label': '70'}, page_content='Llama 2-Chat\n7B 57.04 60.59 96.45\n13B 62.18 65.73 96.45\n34B 67.2 70.01 97.06\n70B 64.14 67.07 97.06\nTable 44: Evaluation results on TruthfulQA across different model generations.\nLimitations of Benchmarks. It is important to note that these evaluations using automatic metrics are by\nno means fully comprehensive, due to the complex nature of toxicity and bias in LLMs, but the benchmarks\nwe selected are representative of our understanding thatLlama 2-C

In [95]:
relevant_results[2].page_content

'descriptions of the benchmarks and metrics can be found in Appendix A.4.7. When compared toLlama 1-7B,\nLlama 2-7B demonstrates a 21.37% increase in truthfulness and informativeness and a 7.61% decrease in\ntoxicity. We also observe an increase in toxicity in the pretrained 13B and 70BLlama 2, which may result\nfrom larger pretraining data or a different dataset mix. Some have postulated the existence of a relationship'

In [96]:
# by default k=4
relevant_results = vector_store.similarity_search("llama2 finetuning benchmark experiments?" ,k=2)
relevant_results

[Document(id='ae06a9b0-3328-4710-a594-a0586a340258', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'e:\\LLMProjects\\documentportal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 69, 'page_label': '70'}, page_content='Llama 2-Chat\n7B 57.04 60.59 96.45\n13B 62.18 65.73 96.45\n34B 67.2 70.01 97.06\n70B 64.14 67.07 97.06\nTable 44: Evaluation results on TruthfulQA across different model generations.\nLimitations of Benchmarks. It is important to note that these evaluations using automatic metrics are by\nno means fully comprehensive, due to the complex nature of toxicity and bias in LLMs, but the benchmarks\nwe selected are representative of our understanding thatLlama 2-C

In [97]:
retriever = vector_store.as_retriever()

In [98]:
retriever.invoke("llama2 finetuning benchmark experiments?")

[Document(id='ae06a9b0-3328-4710-a594-a0586a340258', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'e:\\LLMProjects\\documentportal\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 69, 'page_label': '70'}, page_content='Llama 2-Chat\n7B 57.04 60.59 96.45\n13B 62.18 65.73 96.45\n34B 67.2 70.01 97.06\n70B 64.14 67.07 97.06\nTable 44: Evaluation results on TruthfulQA across different model generations.\nLimitations of Benchmarks. It is important to note that these evaluations using automatic metrics are by\nno means fully comprehensive, due to the complex nature of toxicity and bias in LLMs, but the benchmarks\nwe selected are representative of our understanding thatLlama 2-C

In [99]:
# question - user query
# context - relevant documents retrieved from vector database
# response - answer generated by the LLM

In [100]:
prompt_template = """Answer the question based on the context below.
if the context is not relevant, say 'I don't know'

context: {context}
question: {question}
answer:
"""

In [102]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context","question"]
)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="Answer the question based on the context below.\nif the context is not relevant, say 'I don't know'\n\ncontext: {context}\nquestion: {question}\nanswer:\n")

In [107]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
parser = StrOutputParser()

In [108]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [112]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
#rag_chain


In [113]:
rag_chain.invoke("llama2 finetuning benchmark experiments?")

'<think>\nOkay, let\'s tackle this query. The user is asking about "llama2 finetuning benchmark experiments." First, I need to check the provided context to see if it mentions anything about fine-tuning benchmarks for Llama 2.\n\nLooking at the context, there\'s a table (Table 44) showing evaluation results for Llama 2-Chat across different model sizes on the TruthfulQA benchmark. The context also discusses limitations of benchmarks and mentions other benchmarks like MMLU, BBH, and AGI Eval. However, the main focus is on comparing Llama 2 models with previous versions and other models like GPT-3.5, GPT-4, PaLM, and PaLM-2-L. \n\nThe text talks about improvements in truthfulness, informativeness, and toxicity, but there\'s no specific mention of fine-tuning experiments. The benchmarks discussed are for the base models, not fine-tuned versions. The section about data contamination is in Appendix A.6, but that\'s not detailed here. Since the user is asking about fine-tuning benchmarks and